#### Task1
В качестве разрешенных выбраны следующие кодовые комбинации:<br>
  - *Е1=0000110111*
  - *Е2=0000101011*
  - *Е3=1000011001*
  - *Е4=0010010110*

Определить максимальную кратность обнаруживаемых и исправляемых ошибок.<br>
Показать процесс исправления ошибок максимальной кратности при передаче первой
разрешенной кодовой комбинации.

In [87]:
import numpy as np
from random import randrange

def xor(a, b):
    return (a & (~b)) | ((~a) & b)

# messages with length of 7
msg_len = 10;
E1 = 0b0000110111
E2 = 0b0000101011
E3 = 0b1000011001
E4 = 0b0010010110

# list of messages
msg_list = np.array([0, E1, E2, E3, E4])

msg_list_len = len(msg_list)

# dest array
d = np.array([[0] * msg_list_len] * msg_list_len)

d_list = []

for i in range(1, msg_list_len - 1):
    for j in range(i + 1, msg_list_len):
        d[i][j] = xor(msg_list[i], msg_list[j]).bit_count()
        print(f"d{i}{j} = {d[i][j]}")
        d_list.append(d[i][j])

d_min = min(d_list)
print(f"\nd_min = {d_min}")

# t_ob -- order of error that is able to be detected
print("\nd_min >= t_ob + 1")
t_ob = d_min - 1
print(f"t_ob <= {t_ob}\n")

# t_isp -- order of error that is able to be fixed
print("d_min >= 2*t_isp + 1")
t_isp = (d_min - 1) / 2
print(f"t_isp <= {t_isp}\n")

# any error vector of t_isp order
# error_vec = 0b0 | (1 << randrange(msg_len))
error_vec = 0b1000000000
print(f"error_vec = {bin(error_vec)}\n")

# adding error_vec in msg E1
new_E1 = xor(E1, error_vec)
print(f"new_E1 = E1 ^ error_vec = {bin(new_E1)}")

# check dest from new msg to start msgs
min_dest = msg_len + 1
min_dest_i = 0
for i in range(1, msg_list_len):
    dest = xor(msg_list[i], new_E1).bit_count()
    print(f"d{i} = {dest}")

    if dest < min_dest :
        min_dest = dest
        min_dest_i = i

print(f"min_dest = d{min_dest_i} = {min_dest}")

d12 = 3
d13 = 5
d14 = 3
d23 = 4
d24 = 6
d34 = 6

d_min = 3

d_min >= t_ob + 1
t_ob <= 2

d_min >= 2*t_isp + 1
t_isp <= 1.0

error_vec = 0b1000000000

new_E1 = E1 ^ error_vec = 0b1000110111
d1 = 1
d2 = 4
d3 = 4
d4 = 4
min_dest = d1 = 1
